In [43]:
# Specifying all the imports
from langchain.chat_models import ChatOpenAI
from langchain_core.tools import tool
from langchain.tools import Tool
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.prompts.prompt import PromptTemplate
from langchain.agents import create_react_agent
from langchain.document_loaders import UnstructuredHTMLLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.agents import create_react_agent, AgentExecutor, ZeroShotAgent
from langchain import LLMChain
from dotenv import load_dotenv
import os

In [44]:
# Loading dotenv and specifying tools
load_dotenv()

@tool
def search_tool(query : str):
    """ 
    Takes a user query and performs an online search to provide
    an answer to the query
    """
    
    search = DuckDuckGoSearchRun()
    answer = search.run(query)
    return answer

@tool
def create_embeddings(url):
    """
    Takes a URL or file path and create a vector embedding based on the
    content on the file
    """
    loader = UnstructuredHTMLLoader(url)
    document = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunk = text_splitter.split_documents(documents=document)
    embeddings = OpenAIEmbeddings(api_key=os.getenv('OPENAI_API_KEY'))
    vectorstore = FAISS.from_documents(chunk, embeddings)

    return vectorstore

tools = [Tool(name="search_tool", func=search_tool, description="Tool for performing online search operations"),
         Tool(name="create_embeddings", func=create_embeddings, description="Tool for creating vector embeddings from URL or file path")]

    

In [45]:
template = """
You need to use the tools to find the appropriate action to take. 
This may be answering a question or uploading a file to the vector database.

You have access to the following tools:
{tools}

Question: the input question you must answer

Thought: you should always think about what to do

Action: the action to take, should be one of [{tool_names}]

Action Input: the input to the action

Observation: the result of the action

... (this Thought/Action/Action Input/Observation can repeat N times)

Thought: I now know the final answer

Final Answer: the final answer to the original input question

Begin!

Question: {input}

Thought:{agent_scratchpad}
"""

# Define the query and create the prompt
query = '/mnt/chromeos/MyFiles/Machine_Learning/Individual Machine Learning Projects/Final Project /cr7.html'
prompt = PromptTemplate(input_variables=['input','tools', 'agent_scratchpad', 'tool_names'], template=template)

# Initialize the language model
llm = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model='gpt-3.5-turbo', temperature=0.2)


# Create the agent using the language model and the toolset
agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

# Create the agent executor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

# Execute the agent with the given input
response = agent_executor.invoke({'input': query, 'tools' : tools, 'tool_names' : [tool.name for tool in tools]})

print(response['output'])



> Entering new AgentExecutor chain...
I should use the create_embeddings tool to create vector embeddings from the provided file path.

Action: create_embeddings

Action Input: /mnt/chromeos/MyFiles/Machine_Learning/Individual Machine Learning Projects/Final Project /cr7.html


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/chromeos/MyFiles/Machine_Learning/Individual Machine Learning Projects/Final Project /cr7.html\n'